In [9]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from folium import GeoJson, GeoJsonTooltip
import folium
import geopandas as gpd
from folium.plugins import GroupedLayerControl, MarkerCluster
from folium.plugins import HeatMap
from branca.colormap import linear
import osmnx as ox
from shapely.geometry import LineString, MultiLineString
import overpy
from rasterstats import zonal_stats
from sklearn.preprocessing import MinMaxScaler

import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio import Affine
from rasterio.features import shapes
from shapely.geometry import shape, box, Polygon
import numpy as np
import folium

Ignore warnings

In [10]:
import warnings
warnings.filterwarnings("ignore")

SSL Verification Handling

In [11]:
import certifi
ca_bundle_path = certifi.where() 
print(ca_bundle_path)

c:\Users\Oussama Raji\Anaconda\Lib\site-packages\certifi\cacert.pem


Load Abu Dhabi Shapefile (Admin Level 3)

In [4]:
# 1. Read Abu Dhabi shapefile, filter
shapefile_path = "../Data/UAE_Shapefile/gadm41_ARE_shp/gadm41_ARE_3.shp"
gdf = gpd.read_file(shapefile_path)

# Filter for Abu Dhabi Regions
abu_dhabi_gdf = gdf[gdf["NAME_1"] == "Abu Dhabi"]

# Get the polygon
abu_dhabi_geom = abu_dhabi_gdf.unary_union

Get the Build on Areas Geodata

In [12]:
tif_folder = "../Data/ESA_TIF"
tif_files = [
    os.path.join(root, file)
    for root, dirs, files in os.walk(tif_folder)
    for file in files
    if file.lower().endswith("map.tif")
]

built_up_polygons = []

# Helper to check raster intersection
def raster_intersects(src, geom):
    raster_bounds = box(*src.bounds)
    return geom.intersects(raster_bounds)

# Downsample function
def downsample_raster(image, factor=2):
    return image[::factor, ::factor]

# Simplify polygons
def simplify_geometry(geom, tolerance=0.01):
    if isinstance(geom, Polygon):
        return geom.simplify(tolerance, preserve_topology=True)
    return geom

for tif_path in tif_files:
    with rasterio.open(tif_path) as src:
        # Make sure the raster intersects Abu Dhabi
        if not raster_intersects(src, abu_dhabi_geom):
            continue

        # If src.crs != EPSG:4326, consider reprojecting here or reproject after shape extraction
        # E.g. if you want to do it after shapes, you'll do built_up_gdf.to_crs("EPSG:4326")

        try:
            out_image, out_transform = mask(src, [abu_dhabi_geom], crop=True)
            out_image = out_image[0]

            # Downsample
            factor = 4
            downsampled_image = downsample_raster(out_image, factor=factor)

            # Fix the transform scaling
            downsampled_transform = out_transform * Affine.scale(factor, factor)

            # Extract shapes
            for geom_val, val in shapes(
                    downsampled_image,
                    mask=(downsampled_image == 50),
                    transform=downsampled_transform
            ):
                if val == 50:
                    poly = shape(geom_val)

                    # Filter out tiny polygons if you wish
                    if poly.area < 1e-7:
                        continue

                    # Simplify geometry more aggressively
                    poly_simpl = simplify_geometry(poly, tolerance=0.01)
                    built_up_polygons.append(poly_simpl)

        except Exception as e:
            print(f"⚠️ Failed to process {tif_path}: {e}")

# Convert to a GeoDataFrame
built_up_gdf = gpd.GeoDataFrame(geometry=built_up_polygons, crs="EPSG:4326")  # or the actual CRS if different

In [ ]:
# Create a Build on Areas Folium map
center = abu_dhabi_gdf.geometry.centroid.iloc[0]
m = folium.Map(location=[center.y, center.x], zoom_start=9, tiles='cartodbpositron')

# Add built-up polygons
folium.GeoJson(
    built_up_gdf.__geo_interface__,
    name="Built-Up Areas",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.5}
).add_to(m)

# Add Abu Dhabi boundary
folium.GeoJson(
    abu_dhabi_gdf.__geo_interface__,
    name="Abu Dhabi Boundary",
    style_function=lambda x: {'color': 'blue', 'weight': 2, 'fillOpacity': 0}
).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
#m.save("../Data/Maps/built_up_areas_map_lightweight411Test.html")

Create a Scaled Population Density based on the CENSUS file

In [6]:
# Census data path
census_path = "../Data/Population_Data/Abu_Dhabi_Districts_Population_Census.xlsx"

# Read Excel file file of Census
mapping_districts_df = pd.read_excel(census_path, sheet_name= "Mapping Districts")
mapping_districts_df.head(2)

,Region,Shapefile_Districts,Census_Districts,Population,Scaled_Pop
0,Abu Dhabi,Abu Dhabi Island,Al Nahyan - Al Zahiyah - Jarn Yafour,257715,1.0
1,Abu Dhabi,Al Falah,Al Falah,52775,0.0


In [17]:
# Merge the GeoDataFrame with the mapping dataframe based on matching 'NAME_3' and 'Shapefile_Districts'
abu_dhabi_gdf = abu_dhabi_gdf.merge(mapping_districts_df[['Shapefile_Districts', 'Census_Districts', 'Scaled_Pop']], left_on='NAME_3', right_on='Shapefile_Districts', how='left')

# Drop the unnecessary 'Shapefile_Districts' column if no longer needed
abu_dhabi_gdf = abu_dhabi_gdf.drop(columns=['Shapefile_Districts'])

Calculate the Midpoint Coordinates

In [13]:
# total_bounds → (minx, miny, maxx, maxy) for the entire GeoDataFrame
minx, miny, maxx, maxy = abu_dhabi_gdf.total_bounds

# x corresponds to longitude, y corresponds to latitude
sw_lat = miny  # southernmost lat
sw_lon = minx  # westernmost lon
ne_lat = maxy  # northernmost lat
ne_lon = maxx  # easternmost lon

print("Overall Bounding Box for KSA Polygons:")
print(f"  SW corner (lat, lon): ({sw_lat}, {sw_lon})")
print(f"  NE corner (lat, lon): ({ne_lat}, {ne_lon})")

# Calculate midpoint coordinates
mid_lat = (sw_lat + ne_lat) / 2
mid_lon = (sw_lon + ne_lon) / 2

Overall Bounding Box for KSA Polygons:
  SW corner (lat, lon): (22.631621401000018, 51.49797800000016)
  NE corner (lat, lon): (25.252622605000056, 56.018131256000174)


Get the Roadways Geodata

In [14]:
# Initialize theoverpy API
api = overpy.Overpass()

all_clipped_roads = []  # Will hold a GeoDataFrame for each geometry

for idx, row in abu_dhabi_gdf.iterrows():
    geom = row.geometry  # Polygon or MultiPolygon

    # Get bounding box from that geometry
    minx, miny, maxx, maxy = geom.bounds
    bbox_str = f"{miny},{minx},{maxy},{maxx}"

    # Overpass query: bounding box style
    query = f"""
    (
      way["highway"~"motorway|trunk|primary|secondary|tertiary"]({bbox_str});
    );
    (._;>;);
    out body;
    """

    try:
        # Query Overpass for this bounding box
        result = api.query(query)
    except Exception as e:
        # If any query fails, we log it and continue, so the loop doesn't stop
        print(f"Overpass query failed for geometry index {idx}: {e}")
        continue  # Skip to next geometry

    # Convert Overpass ways to line features
    roads_data = []
    for way in result.ways:
        coords = [(float(node.lon), float(node.lat)) for node in way.nodes]
        if len(coords) >= 2:
            roads_data.append({
                "geometry": LineString(coords),
                "highway": way.tags.get("highway", "unknown"),
                "name": way.tags.get("name", "Unnamed Road"),
                # Optionally keep track of which polygon this road belongs to
                "polygon_idx": idx
            })

    # If there are no roads, skip
    if not roads_data:
        continue

    gdf_roads = gpd.GeoDataFrame(roads_data, crs=abu_dhabi_gdf.crs)

    # Now clip to the exact polygon or multipolygon so we only keep roads inside it
    gdf_roads_clipped = gpd.clip(gdf_roads, geom)

    # Store in a list to merge later
    all_clipped_roads.append(gdf_roads_clipped)

# Merge all results into one big GeoDataFrame
if len(all_clipped_roads) > 0:
    gdf_highways_clipped = pd.concat(all_clipped_roads, ignore_index=True)
    gdf_highways_clipped = gpd.GeoDataFrame(gdf_highways_clipped, crs=abu_dhabi_gdf.crs)
else:
    # If we got nothing, create an empty GeoDataFrame
    gdf_highways_clipped = gpd.GeoDataFrame(columns=["geometry", "highway", "name", "polygon_idx"], crs=abu_dhabi_gdf.crs)

# Remove duplicates if there is overlap in bounding boxes:
gdf_highways_clipped = gdf_highways_clipped.drop_duplicates(subset="geometry")

In [15]:
# Print unique highway types
gdf_highways_clipped["highway"].value_counts(sort= True)

highway
tertiary          6439
secondary         3311
primary           2083
tertiary_link     1470
primary_link      1386
secondary_link    1370
motorway_link     1353
trunk              758
trunk_link         733
motorway           559
Name: count, dtype: int64

In [ ]:
# Create the map centered on the midpoint with appropriate zoom level
m = folium.Map(location=[mid_lat, mid_lon], zoom_start=9, tiles='cartodbpositron')

# Add the GeoJSON polygon to the map
folium.GeoJson(
    data=abu_dhabi_gdf,
    name='Abu Dhabi Boundary',
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_3'],
        aliases=['District: '],
        localize=True
    )
).add_to(m)

for _, row in gdf_highways_clipped.iterrows():
    geom = row.geometry

    # Handle LineString
    if isinstance(geom, LineString):
        coords = [(lat, lon) for lon, lat in geom.coords]
        folium.PolyLine(
            locations=coords,
            color='#ADD8E6',
            weight=2,
            tooltip=row["name"]
        ).add_to(m)

    # Handle MultiLineString
    elif isinstance(geom, MultiLineString):
        for linestring in geom.geoms:
            coords = [(lat, lon) for lon, lat in linestring.coords]
            folium.PolyLine(
                locations=coords,
                color='red',
                weight=2,
                tooltip=row["name"]
            ).add_to(m)


# Save the updated map with roadways
#m.save('../Data/Maps/Abu_Dhabi_Map_with_Roadways411_Search_By_Districts_100.html')

Preprocessing of the scrapped GPA Data

In [106]:
# Import scrapped data from GPA
filename = "../Data/GPA_Data/EncodingData_output.xlsx"
sheet_name="Data"
df = pd.read_excel(filename,sheet_name=sheet_name)
# Preview the first 2 rows
df.head(2)

,name,id,types,atm,auto_parts_store,car_rental,car_repair,car_wash,convenience_store,corporate_office,...,liveMusic,servesDessert,servesCoffee,goodForGroups,priceRange_startPrice_currencyCode,priceRange_startPrice_units,priceRange_endPrice_currencyCode,priceRange_endPrice_units,paymentOptions_acceptsCashOnly,geometry
0,places/ChIJB9fWcYriZj4RfEi5X7IJObk,ChIJB9fWcYriZj4RfEi5X7IJObk,"gas_station, atm, finance, point_of_interest, ...",1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (53.6062302 23.0885438)
1,places/ChIJa2fP583jZj4Rl5NpV3hEf9c,ChIJa2fP583jZj4Rl5NpV3hEf9c,"gas_station, point_of_interest, establishment",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (53.6062314 23.0887015)


In [91]:
# Get unique types
unique_types = df["types"].unique().tolist()
unique_types[:10]

['gas_station, atm, finance, point_of_interest, establishment',
 'gas_station, point_of_interest, establishment',
 'gas_station, atm, car_wash, convenience_store, bakery, coffee_shop, cafe, car_repair, finance, food_store, food, point_of_interest, store, establishment',
 'gas_station, mosque, atm, deli, convenience_store, dessert_shop, bakery, coffee_shop, cafe, place_of_worship, finance, confectionery, food_store, food, point_of_interest, store, establishment',
 'gas_station, coffee_shop, cafe, food_store, food, point_of_interest, store, establishment',
 'gas_station, atm, convenience_store, car_wash, bakery, coffee_shop, cafe, car_repair, finance, food_store, store, food, point_of_interest, establishment',
 'gas_station, mosque, place_of_worship, point_of_interest, establishment',
 'gas_station, coffee_shop, cafe, food_store, store, food, point_of_interest, establishment',
 'gas_station, deli, convenience_store, car_wash, bakery, coffee_shop, cafe, car_repair, food_store, store, food

In [92]:
# Split by ',' to get all types
all_types = []
for type in unique_types:
    if "," in type:
        all_types.extend(type.split(","))
    else:
        all_types.append(type)
print(all_types)

['gas_station', ' atm', ' finance', ' point_of_interest', ' establishment', 'gas_station', ' point_of_interest', ' establishment', 'gas_station', ' atm', ' car_wash', ' convenience_store', ' bakery', ' coffee_shop', ' cafe', ' car_repair', ' finance', ' food_store', ' food', ' point_of_interest', ' store', ' establishment', 'gas_station', ' mosque', ' atm', ' deli', ' convenience_store', ' dessert_shop', ' bakery', ' coffee_shop', ' cafe', ' place_of_worship', ' finance', ' confectionery', ' food_store', ' food', ' point_of_interest', ' store', ' establishment', 'gas_station', ' coffee_shop', ' cafe', ' food_store', ' food', ' point_of_interest', ' store', ' establishment', 'gas_station', ' atm', ' convenience_store', ' car_wash', ' bakery', ' coffee_shop', ' cafe', ' car_repair', ' finance', ' food_store', ' store', ' food', ' point_of_interest', ' establishment', 'gas_station', ' mosque', ' place_of_worship', ' point_of_interest', ' establishment', 'gas_station', ' coffee_shop', ' ca

In [103]:
# Get the Mapping Dict Data
filename = "../Data/GPA_Data/EncodingData_output.xlsx"
sheet_name="Type_Mapping"
type_mapping = pd.read_excel(filename,sheet_name=sheet_name)
type_mapping[type_mapping['Mapping'].isna() | (type_mapping['Mapping'] == "")].head()

,types,Mapping
29,establishment,NaN
30,point_of_interest,NaN


In [104]:
# Filter rows where 'Mapping' is NaN or empty string
to_beremoved_types = type_mapping[type_mapping['Mapping'].isna() | (type_mapping['Mapping'] == "")]["types"].tolist()
to_beremoved_types

['establishment', 'point_of_interest']

In [105]:
# Create a dict to map the types (based o Hadia Mapping)
mapping_dict  = dict(zip(type_mapping['types'], type_mapping['Mapping']))
print(mapping_dict )

{'gas_station': 'gas_station', 'car_repair': 'car_repair', 'corporate_office': 'corporate_office', 'coffee_shop': 'food_shop', 'convenience_store': 'convenience_store', 'atm': 'atm', 'sandwich_shop': 'food_shop', 'public_bathroom': 'public_bathroom', 'auto_parts_store': 'auto_parts_store', 'car_wash': 'car_wash', 'mosque': 'place_of_worship', 'food_court': 'food_restaurant', 'fast_food_restaurant': 'food_restaurant', 'dessert_shop': 'food_shop', 'deli': 'food_shop', 'finance': 'finance', 'cafe': 'food_restaurant', 'electric_vehicle_charging_station': 'electric_vehicle_charging_station', 'bakery': 'food_shop', 'store': 'convenience_store', 'food_store': 'food_shop', 'supermarket': 'convenience_store', 'food': 'food_shop', 'place_of_worship': 'place_of_worship', 'confectionery': 'food_shop', 'grocery_store': 'convenience_store', 'restaurant': 'food_restaurant', 'meal_takeaway': 'food_shop', 'car_rental': 'car_rental', 'establishment': nan, 'point_of_interest': nan}


In [ ]:
# Function to map types in the list to their corresponding Mapping values
def map_types(types_string):
    if isinstance(types_string, str):  # Check if the input is a string
        types_list = types_string.split(', ')  # Split the string into a list
        mapped_types = [mapping_dict.get(t, 'nan') for t in types_list]  # Map each type, return None if no match
        # Remove 'None' values (equivalent to NaN in this case)
        mapped_types = [t for t in mapped_types if t is not 'nan']
        mapped_types = [str(t) for t in mapped_types]  # Ensure all items are strings before joining
        return ', '.join([item for item in mapped_types if item != 'nan'])
    return ''  # Return an empty string if types_string is not a valid string

# Apply the function to the 'types' column in your df
#del df['Mapped_Types']
df['Mapped_Types'] = df['types'].apply(map_types)

Brand Mapping {'ADNOC', 'ENOC', 'Others'}

In [117]:
# Group by "displayName_text" {'ADNOC', 'ENOC', 'Others'}

# ADNOC: include English, Arabic, loose patterns
regex_adnoc = r'(?i)\b(adnoc|ادنوك|محطة\s*ادنوك|adnoc\s*(service\s*station|lube|cng|vehicle inspection)|aldhfra petrol pump adnoc)\b'

# ENOC: include English and Arabic potential
regex_enoc = r'(?i)\b(enoc|اينوك|محطة\s*اينوك)\b'

# ADNOC DataFrame
df_adnoc = df[df['displayName_text'].str.contains(regex_adnoc, na=False)]

# ENOC DataFrame
df_enoc = df[df['displayName_text'].str.contains(regex_enoc, na=False)]

# Others DataFrame
matched_indexes = pd.concat([df_adnoc, df_enoc]).index
df_others = df[~df.index.isin(matched_indexes)]

Create an overall Map including all layers (Build on areas - Roadways - Pop Density Scaled - Gas Station)

In [121]:
# Initialize map
m = folium.Map(location=[mid_lat, mid_lon], zoom_start=9, tiles='cartodbpositron')

# Add GeoJSON with fill and border color
region_colors = {
    'Abu Dhabi': '#673400',  # Dark Brown
    'Al Ain':    '#964B00',  # Reddish Brown
    'Al Dhafra': '#B5651D'   # Beige / Sand
}

# Add built-up areas
folium.GeoJson(
    built_up_gdf.__geo_interface__,
    name="Built-up Areas",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.5}
).add_to(m)

# Create a color map (yellow to red) based on the Scaled_Pop
colormap = linear.YlOrRd_09.scale(0, 1)
colormap.caption = 'Population Density'
pop_density_layer = folium.FeatureGroup(name='Population Density', show=True)
folium.GeoJson(
    abu_dhabi_gdf,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Scaled_Pop']) if feature['properties']['Scaled_Pop'] is not None else '#ededed',
        'color': 'none',  # No border for this layer
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_2', 'Scaled_Pop'],
        aliases=['Region: ', 'Scaled Population: '],
        localize=True
    )
).add_to(pop_density_layer)
pop_density_layer.add_to(m)

# Create a separate FeatureGroup for region boundaries
region_boundaries_layer = folium.FeatureGroup(name='Region Boundaries', show=True)
folium.GeoJson(
    abu_dhabi_gdf,
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': region_colors.get(feature['properties']['NAME_2'].strip(), '#ededed'),
        'weight': 2,
        'fillOpacity': 0.9
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_2'],
        aliases=['Region: '],
        localize=True
    )
).add_to(region_boundaries_layer)
region_boundaries_layer.add_to(m)



# Add clustered markers for ADNOC, ENOC, Others
df_map = {
    "ADNOC": {"df": df_adnoc, "color": "blue"},
    "ENOC": {"df": df_enoc, "color": "green"},
    "Others": {"df": df_others, "color": "grey"},
}

## Add gas stations and brand clustering
for brand, props in df_map.items():
    df = props["df"]
    color = props["color"]
    feature_group = folium.FeatureGroup(name=brand, show=True).add_to(m)

    marker_cluster = MarkerCluster(
        icon_create_function=f"""
        function(cluster) {{
            return L.divIcon({{
                html: '<div style="background-color: {color}; color: white; border-radius: 50%; width: 30px; height: 30px; line-height: 30px; text-align: center; font-weight: bold; font-size: 12px;">' + cluster.getChildCount() + '</div>',
                className: 'marker-cluster',
                iconSize: new L.Point(30, 30)
            }});
        }}
        """
    ).add_to(feature_group)

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row['location_latitude'], row['location_longitude']],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            # I should add the translated name based on Souleiman Output
            popup=folium.Popup(f"""
                <b>Name:</b> {row['displayName_text']}<br>
                <b>Address:</b> {row.get('formattedAddress', 'No Address')}<br>
                <b>Longitude:</b> {row['location_longitude']}<br>
                <b>Latitude:</b> {row['location_latitude']}<br>
                <b>Rating:</b> {row.get('rating', 'No Ratings')}<br>
                <b>Services:</b> {row['Mapped_Types']}<br>
                <b>Link:</b> <a href="{row.get('googleMapsUri', 'No Link Provided')}" target="_blank">View on Google Maps</a>
            """, max_width=300)
        ).add_to(marker_cluster)

# Add Roadways
for _, row in gdf_highways_clipped.iterrows():
    geom = row.geometry

    # Handle LineString
    if isinstance(geom, LineString):
        coords = [(lat, lon) for lon, lat in geom.coords]
        folium.PolyLine(
            locations=coords,
            color='#ADD8E6',
            weight=2,
            tooltip=row["name"]
        ).add_to(m)

    # Handle MultiLineString
    elif isinstance(geom, MultiLineString):
        for linestring in geom.geoms:
            coords = [(lat, lon) for lon, lat in linestring.coords]
            folium.PolyLine(
                locations=coords,
                color='#ADD8E6',
                weight=2,
                tooltip=row["name"]
            ).add_to(m)


# Add legend
legend_html = '''
<div style="
    position: fixed;
    top: 10px;
    left: 10px;
    width: 180px;
    background-color: white;
    border: 2px solid gray;
    z-index: 9999;
    padding: 10px;
    font-size: 14px;
    font-family: Arial, sans-serif;">
    <b>Gas Stations</b><br>
'''

for brand, props in df_map.items():
    legend_html += f'''
    <span style="
        display: inline-block;
        width: 12px;
        height: 12px;
        margin-right: 6px;
        border-radius: 50%;
        background-color: {props['color']};">
    </span> {brand}<br>
    '''

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))


# Add region boundaries legend below the gas station legend
legend_region_colors = '''
<div id="legend-region" style="
    position: fixed;
    top: 150px;  /* Adjust this value as needed to ensure proper spacing below the gas legend */
    left: 10px;
    width: 180px;
    background-color: white;
    border: 2px solid gray;
    z-index: 9999;
    padding: 10px;
    font-size: 14px;
    font-family: Arial, sans-serif;">
    <b>Region Boundaries</b><br>
'''

for region, color in region_colors.items():
    legend_region_colors += f'''
    <span style="
        display: inline-block;
        width: 12px;
        height: 12px;
        margin-right: 6px;
        background-color: {color};">
    </span> {region}<br>
    '''
legend_region_colors += '</div>'
m.get_root().html.add_child(folium.Element(legend_region_colors))

# Add color scale + layer control
colormap.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# ---- Save map ----
m.save("../Data/Maps/Gas_Stations_Abu_Dhabi_City_all_layers_VF_test2.html")

Draft

In [ ]:
# Initialize map
m = folium.Map(location=[mid_lat, mid_lon], zoom_start=9, tiles='cartodbpositron')

# Add GeoJSON with fill and border color
region_colors = {
    'Abu Dhabi': '#3C1F18',  # Dark Brown
    'Al Ain':    '#983F27',  # Reddish Brown
    'Al Dhafra': '#EAD4B1'   # Beige / Sand
}

# Add built-up areas
folium.GeoJson(
    built_up_gdf.__geo_interface__,
    name="Built-up Areas",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.5}
).add_to(m)

# Create a color map (yellow to red) based on the Scaled_Pop
colormap = linear.YlOrRd_09.scale(0, 1)
colormap.caption = 'Population Density'
pop_density_layer = folium.FeatureGroup(name='Population Density', show=True)
folium.GeoJson(
    abu_dhabi_gdf,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Scaled_Pop']) if feature['properties']['Scaled_Pop'] is not None else '#ededed',
        'color': 'none',  # No border for this layer
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_2', 'Scaled_Pop'],
        aliases=['Region: ', 'Scaled Population: '],
        localize=True
    )
).add_to(pop_density_layer)
pop_density_layer.add_to(m)

# Create a separate FeatureGroup for region boundaries
region_boundaries_layer = folium.FeatureGroup(name='Region Boundaries', show=True)
folium.GeoJson(
    abu_dhabi_gdf,
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': region_colors.get(feature['properties']['NAME_2'].strip(), '#ededed'),
        'weight': 2,
        'fillOpacity': 0.9
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_2'],
        aliases=['Region: '],
        localize=True
    )
).add_to(region_boundaries_layer)
region_boundaries_layer.add_to(m)



# Add clustered markers for ADNOC, ENOC, Others
df_map = {
    "ADNOC": {"df": df_adnoc, "color": "blue"},
    "ENOC": {"df": df_enoc, "color": "green"},
    "Others": {"df": df_others, "color": "grey"},
}

## Add gas stations and brand clustering
for brand, props in df_map.items():
    df = props["df"]
    color = props["color"]
    feature_group = folium.FeatureGroup(name=brand, show=True).add_to(m)

    marker_cluster = MarkerCluster(
        icon_create_function=f"""
        function(cluster) {{
            return L.divIcon({{
                html: '<div style="background-color: {color}; color: white; border-radius: 50%; width: 30px; height: 30px; line-height: 30px; text-align: center; font-weight: bold; font-size: 12px;">' + cluster.getChildCount() + '</div>',
                className: 'marker-cluster',
                iconSize: new L.Point(30, 30)
            }});
        }}
        """
    ).add_to(feature_group)

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row['location_latitude'], row['location_longitude']],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            # I should add the translated name based on Souleiman Output
            popup=folium.Popup(f"""
                <b>Name:</b> {row['displayName_text']}<br>
                <b>Address:</b> {row.get('formattedAddress', 'No Address')}<br>
                <b>Longitude:</b> {row['location_longitude']}<br>
                <b>Latitude:</b> {row['location_latitude']}<br>
                <b>Rating:</b> {row.get('rating', 'No Ratings')}<br>
                <b>Services:</b> {row["types"].replace("_", " ")}<br>
                <b>Link:</b> <a href="{row.get('googleMapsUri', 'No Link Provided')}" target="_blank">View on Google Maps</a>
            """, max_width=300)
        ).add_to(marker_cluster)

# Add Roadways
for _, row in gdf_highways_clipped.iterrows():
    geom = row.geometry

    # Handle LineString
    if isinstance(geom, LineString):
        coords = [(lat, lon) for lon, lat in geom.coords]
        folium.PolyLine(
            locations=coords,
            color='#ADD8E6',
            weight=2,
            tooltip=row["name"]
        ).add_to(m)

    # Handle MultiLineString
    elif isinstance(geom, MultiLineString):
        for linestring in geom.geoms:
            coords = [(lat, lon) for lon, lat in linestring.coords]
            folium.PolyLine(
                locations=coords,
                color='#ADD8E6',
                weight=2,
                tooltip=row["name"]
            ).add_to(m)


# Add legend
legend_html = '''
<div style="
    position: fixed;
    top: 10px;
    left: 10px;
    width: 180px;
    background-color: white;
    border: 2px solid gray;
    z-index: 9999;
    padding: 10px;
    font-size: 14px;
    font-family: Arial, sans-serif;">
    <b>Gas Stations</b><br>
'''

for brand, props in df_map.items():
    legend_html += f'''
    <span style="
        display: inline-block;
        width: 12px;
        height: 12px;
        margin-right: 6px;
        border-radius: 50%;
        background-color: {props['color']};">
    </span> {brand}<br>
    '''

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))


# Add region boundaries legend below the gas station legend
legend_region_colors = '''
<div id="legend-region" style="
    position: fixed;
    top: 150px;  /* Adjust this value as needed to ensure proper spacing below the gas legend */
    left: 10px;
    width: 180px;
    background-color: white;
    border: 2px solid gray;
    z-index: 9999;
    padding: 10px;
    font-size: 14px;
    font-family: Arial, sans-serif;">
    <b>Region Boundaries</b><br>
'''

for region, color in region_colors.items():
    legend_region_colors += f'''
    <span style="
        display: inline-block;
        width: 12px;
        height: 12px;
        margin-right: 6px;
        background-color: {color};">
    </span> {region}<br>
    '''
legend_region_colors += '</div>'
m.get_root().html.add_child(folium.Element(legend_region_colors))





# Add color scale + layer control
colormap.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# ---- Save map ----
m.save("../Data/Maps/Gas_Stations_Abu_Dhabi_City_all_layers_VF_test2.html")

In [ ]:
# Initialize map
m = folium.Map(location=[mid_lat, mid_lon], zoom_start=9, tiles='cartodbpositron')

# Add GeoJSON with fill and border color
region_colors = {
    'Abu Dhabi': '#3C1F18',  # Dark Brown
    'Al Ain':    '#983F27',  # Reddish Brown
    'Al Dhafra': '#EAD4B1'   # Beige / Sand
}

# Add built-up polygons
folium.GeoJson(
    built_up_gdf.__geo_interface__,
    name="Built-up Areas",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.5}
).add_to(m)

# Create a color map (yellow to red) based on the Scaled_Pop
colormap = linear.YlOrRd_09.scale(0, 1)
colormap.caption = 'Population Density'

folium.GeoJson(
    abu_dhabi_gdf,
    name='Abu Dhabi Boundary',
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Scaled_Pop']) if feature['properties']['Scaled_Pop'] is not None else '#ededed',
        'color': region_colors.get(feature['properties']['NAME_2'].strip(), '#ededed'),
        'weight': 2,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_2', 'Scaled_Pop'],
        aliases=['Region: ', 'Scaled Population: '],
        localize=True
    )
).add_to(m)

# Add clustered markers for ADNOC, ENOC, Others
df_map = {
    "ADNOC": {"df": df_adnoc, "color": "blue"},
    "ENOC": {"df": df_enoc, "color": "green"},
    "Others": {"df": df_others, "color": "grey"},
}

## Add gas stations and brand clustering
for brand, props in df_map.items():
    df = props["df"]
    color = props["color"]
    feature_group = folium.FeatureGroup(name=brand, show=True).add_to(m)

    marker_cluster = MarkerCluster(
        icon_create_function=f"""
        function(cluster) {{
            return L.divIcon({{
                html: '<div style="background-color: {color}; color: white; border-radius: 50%; width: 30px; height: 30px; line-height: 30px; text-align: center; font-weight: bold; font-size: 12px;">' + cluster.getChildCount() + '</div>',
                className: 'marker-cluster',
                iconSize: new L.Point(30, 30)
            }});
        }}
        """
    ).add_to(feature_group)

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row['location_latitude'], row['location_longitude']],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            # I should add the translated name based on Souleiman Output
            popup=folium.Popup(f"""
                <b>Name:</b> {row['displayName_text']}<br>
                <b>Address:</b> {row.get('formattedAddress', 'No Address')}<br>
                <b>Longitude:</b> {row['location_longitude']}<br>
                <b>Latitude:</b> {row['location_latitude']}<br>
                <b>Rating:</b> {row.get('rating', 'No Ratings')}<br>
                <b>Services:</b> {row["types"].replace("_", " ")}<br>
                <b>Link:</b> <a href="{row.get('googleMapsUri', 'No Link Provided')}" target="_blank">View on Google Maps</a>
            """, max_width=300)
        ).add_to(marker_cluster)

# Add Roadways
for _, row in gdf_highways_clipped.iterrows():
    geom = row.geometry

    # Handle LineString
    if isinstance(geom, LineString):
        coords = [(lat, lon) for lon, lat in geom.coords]
        folium.PolyLine(
            locations=coords,
            color='#ADD8E6',
            weight=2,
            tooltip=row["name"]
        ).add_to(m)

    # Handle MultiLineString
    elif isinstance(geom, MultiLineString):
        for linestring in geom.geoms:
            coords = [(lat, lon) for lon, lat in linestring.coords]
            folium.PolyLine(
                locations=coords,
                color='#ADD8E6',
                weight=2,
                tooltip=row["name"]
            ).add_to(m)


# Add legend
legend_html = '''
<div style="
    position: fixed;
    top: 10px;
    left: 10px;
    width: 180px;
    background-color: white;
    border: 2px solid gray;
    z-index: 9999;
    padding: 10px;
    font-size: 14px;
    font-family: Arial, sans-serif;">
    <b>Gas Stations</b><br>
'''

for brand, props in df_map.items():
    legend_html += f'''
    <span style="
        display: inline-block;
        width: 12px;
        height: 12px;
        margin-right: 6px;
        border-radius: 50%;
        background-color: {props['color']};">
    </span> {brand}<br>
    '''

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))


# Add region boundaries legend below the gas station legend
legend_region_colors = '''
<div id="legend-region" style="
    position: fixed;
    top: 150px;  /* Adjust this value as needed to ensure proper spacing below the gas legend */
    left: 10px;
    width: 180px;
    background-color: white;
    border: 2px solid gray;
    z-index: 9999;
    padding: 10px;
    font-size: 14px;
    font-family: Arial, sans-serif;">
    <b>Region Boundaries</b><br>
'''

for region, color in region_colors.items():
    legend_region_colors += f'''
    <span style="
        display: inline-block;
        width: 12px;
        height: 12px;
        margin-right: 6px;
        background-color: {color};">
    </span> {region}<br>
    '''
legend_region_colors += '</div>'
m.get_root().html.add_child(folium.Element(legend_region_colors))





# Add color scale + layer control
colormap.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# ---- Save map ----
m.save("../Data/Maps/Gas_Stations_Abu_Dhabi_City_all_layers_VF_test1.html")

In [ ]:
# Initialize map
m = folium.Map(location=[mid_lat, mid_lon], zoom_start=9, tiles='cartodbpositron')

# Add GeoJSON with fill and border color
region_colors = {
    'Abu Dhabi': '#3C1F18',  # Dark Brown
    'Al Ain':    '#983F27',  # Reddish Brown
    'Al Dhafra': '#EAD4B1'   # Beige / Sand
}

## TIF LAYER HERE
# Add built-up polygons
folium.GeoJson(
    built_up_gdf.__geo_interface__,
    name="Built-Up Areas",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.5}
).add_to(m)

# Create a color map (yellow to red) based on the Scaled_Pop
colormap = linear.YlOrRd_09.scale(0, 1)
colormap.caption = 'Population Density'

folium.GeoJson(
    abu_dhabi_gdf,
    name='Abu Dhabi Boundary',
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Scaled_Pop']) if feature['properties']['Scaled_Pop'] is not None else '#ededed',
        'color': region_colors.get(feature['properties']['NAME_2'].strip(), '#ededed'),
        'weight': 2,
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_2', 'Scaled_Pop'],
        aliases=['Region: ', 'Scaled Population: '],
        localize=True
    )
).add_to(m)

# Add clustered markers for ADNOC, ENOC, Others
df_map = {
    "ADNOC": {"df": df_adnoc, "color": "blue"},
    "ENOC": {"df": df_enoc, "color": "green"},
    "Others": {"df": df_others, "color": "grey"},
}

## Add gas stations and brand clustering
for brand, props in df_map.items():
    df = props["df"]
    color = props["color"]
    feature_group = folium.FeatureGroup(name=brand, show=True).add_to(m)

    marker_cluster = MarkerCluster(
        icon_create_function=f"""
        function(cluster) {{
            return L.divIcon({{
                html: '<div style="background-color: {color}; color: white; border-radius: 50%; width: 30px; height: 30px; line-height: 30px; text-align: center; font-weight: bold; font-size: 12px;">' + cluster.getChildCount() + '</div>',
                className: 'marker-cluster',
                iconSize: new L.Point(30, 30)
            }});
        }}
        """
    ).add_to(feature_group)

    for _, row in df.iterrows():
        folium.CircleMarker(
            location=[row['location_latitude'], row['location_longitude']],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup=folium.Popup(f"""
                <b>Name:</b> {row['displayName_text']}<br>
                <b>Address:</b> {row.get('formattedAddress', 'No Address')}<br>
                <b>Longitude:</b> {row['location_longitude']}<br>
                <b>Latitude:</b> {row['location_latitude']}<br>
                <b>Rating:</b> {row.get('rating', 'No Ratings')}<br>
                <b>ATM:</b> {"True" if row["atm"] else "False"}<br>
                <b>Auto Parts Store:</b> {"True" if row["auto_parts_store"] else "False"}<br>
                <b>Car Rental:</b> {"True" if row["car_rental"] else "False"}<br>
                <b>Car Repair:</b> {"True" if row["car_repair"] else "False"}<br>
                <b>Car Wash:</b> {"True" if row["car_wash"] else "False"}<br>
                <b>Convenience Store:</b> {"True" if row["convenience_store"] else "False"}<br>
                <b>Corporate Office:</b> {"True" if row["corporate_office"] else "False"}<br>
                <b>Electric Vehicle Charging:</b> {"True" if row["electric_vehicle_charging_station"] else "False"}<br>
                <b>Establishment:</b> {"True" if row["establishment"] else "False"}<br>
                <b>Finance:</b> {"True" if row["finance"] else "False"}<br>
                <b>Food Restaurant:</b> {"True" if row["food_restaurant"] else "False"}<br>
                <b>Food Shop:</b> {"True" if row["food_shop"] else "False"}<br>
                <b>Gas Station:</b> {"True" if row["gas_station"] else "False"}<br>
                <b>Place of Worship:</b> {"True" if row["place_of_worship"] else "False"}<br>
                <b>Point of Interest:</b> {"True" if row["point_of_interest"] else "False"}<br>
                <b>Public Bathroom:</b> {"True" if row["public_bathroom"] else "False"}<br>
                <b>Link:</b> <a href="{row.get('googleMapsUri', 'No Link Provided')}" target="_blank">View on Google Maps</a>
            """, max_width=300)
        ).add_to(marker_cluster)

# Add Roadways
for _, row in gdf_highways_clipped.iterrows():
    geom = row.geometry

    # Handle LineString
    if isinstance(geom, LineString):
        coords = [(lat, lon) for lon, lat in geom.coords]
        folium.PolyLine(
            locations=coords,
            color='#ADD8E6',
            weight=2,
            tooltip=row["name"]
        ).add_to(m)

    # Handle MultiLineString
    elif isinstance(geom, MultiLineString):
        for linestring in geom.geoms:
            coords = [(lat, lon) for lon, lat in linestring.coords]
            folium.PolyLine(
                locations=coords,
                color='#ADD8E6',
                weight=2,
                tooltip=row["name"]
            ).add_to(m)


# Add legend
legend_html = '''
<div style="
    position: fixed;
    top: 10px;
    left: 10px;
    width: 180px;
    background-color: white;
    border: 2px solid gray;
    z-index: 9999;
    padding: 10px;
    font-size: 14px;
    font-family: Arial, sans-serif;">
    <b>Gas Stations</b><br>
'''

for brand, props in df_map.items():
    legend_html += f'''
    <span style="
        display: inline-block;
        width: 12px;
        height: 12px;
        margin-right: 6px;
        border-radius: 50%;
        background-color: {props['color']};">
    </span> {brand}<br>
    '''

legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))


# Add region boundaries legend below the gas station legend
legend_region_colors = '''
<div id="legend-region" style="
    position: fixed;
    top: 150px;  /* Adjust this value as needed to ensure proper spacing below the gas legend */
    left: 10px;
    width: 180px;
    background-color: white;
    border: 2px solid gray;
    z-index: 9999;
    padding: 10px;
    font-size: 14px;
    font-family: Arial, sans-serif;">
    <b>Region Boundaries</b><br>
'''

for region, color in region_colors.items():
    legend_region_colors += f'''
    <span style="
        display: inline-block;
        width: 12px;
        height: 12px;
        margin-right: 6px;
        background-color: {color};">
    </span> {region}<br>
    '''
legend_region_colors += '</div>'
m.get_root().html.add_child(folium.Element(legend_region_colors))





# Add color scale + layer control
colormap.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# ---- Save map ----
m.save("../Data/Maps/Gas_Stations_Abu_Dhabi_City_all_layers_VF_test1.html")